**Q. Implement a program on Adversarial training, tangent distance, tangent prop and tangent classifier. [Any three to be implemented].**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def generate_adversarial_example(model, x, y, epsilon=0.1):
    x.requires_grad = True
    output = model(x)
    loss = F.cross_entropy(output, y)
    model.zero_grad()
    loss.backward(retain_graph=True)
    x_adv = x + epsilon * x.grad.sign()
    return x_adv

def tangent_distance(x1, x2, model):

    x1.requires_grad = True
    x2.requires_grad = True
    output1 = model(x1)
    output2 = model(x2)
    grad1 = torch.autograd.grad(outputs=output1, inputs=x1, grad_outputs=torch.ones_like(output1), create_graph=True)[0]
    grad2 = torch.autograd.grad(outputs=output2, inputs=x2, grad_outputs=torch.ones_like(output2), create_graph=True)[0]
    dist = torch.norm(grad1 - grad2)
    return dist

def tangent_prop(model, x, y, learning_rate=0.001, epsilon=0.1):
    x_adv = generate_adversarial_example(model, x, y, epsilon)

    x_adv = x_adv.detach().requires_grad_(True)
    output_adv = model(x_adv)
    loss = F.cross_entropy(output_adv, y)
    model.zero_grad()
    loss.backward(retain_graph=True)
    for param in model.parameters():
        param.data -= learning_rate * param.grad.data
    return model, loss

def train_model(model, train_loader, epochs=5):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.view(-1, 28*28), target
            data, target = Variable(data), Variable(target)

            x_adv = generate_adversarial_example(model, data, target)

            model, loss = tangent_prop(model, data, target)

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = SimpleNN()
train_model(model, train_loader, epochs=5)


Epoch [1/5], Loss: 0.5123910307884216
Epoch [2/5], Loss: 0.3119443953037262
Epoch [3/5], Loss: 0.2703799903392792
Epoch [4/5], Loss: 0.16539806127548218
Epoch [5/5], Loss: 0.46896031498908997
